## Visual Analytics 
### Small web scraping for the final Project 
##### Supervisors:  Sippo Rossi & Ivanna Jurkiv 

In [1]:
#importing necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from collections import Counter
import re

In [2]:
#creating link to scrape
link = "http://db-sandsteinklettern.gipfelbuch.de/gipfel.php?sektorid="

#defining the range of pages IDs to scrape
numbers = list(range(123,136))
df = pd.DataFrame()

#looping through the pages
for number in numbers:
    new_link = link + str(number) #add number to link
    ht = requests.get(new_link) #GET request
    soup = BeautifulSoup(ht.text) #paring trough htlm document
    table = soup.find("table") #looking for the table in the htlm

    rows=list() 
    for row in table.findAll("tr"): #iterating over rows in table
        rows.append(row) #appending them
    imgs = {}

    for row in rows: #iterating over appended rows
        string =str(row)
        string = string.replace('"',"")
        match = re.search(r'<font size=>(\d{2,3}G[A-Z]?)</font>', string) #search for this part in the html
        if match:
            extracted_part = match.group(1) #if match, extract this part

    for row in rows[2:]:
        s = str(row)
        gid = row.find_all('a')[0].text #getting the name of summit
        imgs[gid] = Counter([x.get("src") for x in row.find_all("img")]) #getting count of gifs

    new_df = pd.DataFrame.from_dict(imgs, orient='index') #create DataFrame from dictionary
    new_df = new_df.reset_index() 
    new_df = new_df.rename(columns={'index':'summit_name'}) #rename columns
    new_df = new_df.fillna(0) #fill missing values with 0
    df = pd.concat([df, new_df], ignore_index=True) #concatenate dataframes

print(df)

         summit_name  krankenh.gif  gipfelm.gif  gipfeln.gif  quacke.gif  \
0           Hallodri           2.0          0.0          0.0         0.0   
1        Lehnwächter           1.0          0.0          1.0         0.0   
2         Lorenzwand           3.0          1.0          0.0         0.0   
3        Lorenznadel           2.0          0.0          1.0         0.0   
4          Domkanzel           1.0          0.0          0.0         0.0   
..               ...           ...          ...          ...         ...   
884  Hausbergwächter           0.0          0.0          1.0         0.0   
885   Großsteinnadel           0.0          0.0          1.0         0.0   
886  Teichsteinnadel           0.0          0.0          1.0         0.0   
887    E-Flügel-Wand           0.0          0.0          1.0         0.0   
888       Buschmühle           0.0          0.0          0.0         0.0   

     gipfelgr.gif  kreuz.gif  muell.gif  
0             0.0        0.0        NaN  
1  

In [3]:
#cleaning summit_name
#strip trailing characters
df["summit_name"] = df["summit_name"].str.rstrip(" /.")

#split and select first element
df["summit_name"] = df["summit_name"].str.split("/").str[0]

#strip trailing spaces
df["summit_name"] = df["summit_name"].str.rstrip(" ")

In [4]:
#select only specified columns
df = df[["summit_name","kreuz.gif","krankenh.gif"]]

#rename columns
df.rename(columns={'kreuz.gif': 'deaths_on_summit', 'krankenh.gif': 'accidents'}, inplace=True)

#save to csv file
df.to_csv("gif_count.csv", index=False)